In [1]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
data = datasets.load_breast_cancer()
X, y = data.data, data.target

In [3]:
n_samples, n_features = X.shape

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [6]:
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

In [12]:
y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

In [13]:
class LogisticModel(nn.Module):
    def __init__(self, n_input_features):
        super(LogisticModel, self).__init__()
        self.linear = nn.Linear(n_input_features, 1) #binary class

    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred

model = LogisticModel(n_features)
model

LogisticModel(
  (linear): Linear(in_features=30, out_features=1, bias=True)
)

In [14]:
num_epochs = 100
learning_rate = 0.01
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


In [15]:
for epoch in range(num_epochs):

    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if (epoch+1) % 10 == 0:
        print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')


epoch: 10, loss = 0.4434
epoch: 20, loss = 0.3876
epoch: 30, loss = 0.3487
epoch: 40, loss = 0.3201
epoch: 50, loss = 0.2980
epoch: 60, loss = 0.2803
epoch: 70, loss = 0.2658
epoch: 80, loss = 0.2536
epoch: 90, loss = 0.2432
epoch: 100, loss = 0.2342


In [16]:
with torch.no_grad():
    y_pred= model(X_test)
    y_pred_class = y_pred.round()
    acc= y_pred_class.eq(y_test).sum()/float(y_test.shape[0])
  print(f'accuracy: {acc.item():.4f}')

accuracy: 0.9474
